# Dónde estamos?

### Ya sabemos generar sonidos en Python!

### Sonido digital = secuencia de números (muestras o samples)


<br><br>


<center>
<img src="media/noise-code.png" width=80% />
</center>

<br><br>



<center>
<img src="media/sin-code.png" width=80% />
</center>



## Solo nos falta que suenen...

<center>
<img src="media/pyaudio.png" width=80% />
</center>





### Nos interesará:


- <font color='darkgreen'>Grabar</font>: recoger el sonido
    (**muestrear**, ADC) con una tarjeta de sonido y almacenarlo en formato digital

- <font color='darkgreen'>Reproducir</font>: enviar las muestras a la tarjeta de sonido (DAC)

- <font color='darkgreen'>Generar</font>: producir muestras mediante algoritmos $\leadsto$ síntesis digital de sonido


- <font color='darkgreen'>Procesar</font>: transformar las muestras mediante algoritmos (DSP, mezcla)



##### Para ello utilizaremos las librerías: 

- numpy: arrays eficientes no utilizar listas Python para las muestras!
  
  - scipy: para algoritmos específicos de procesamiento de audio

- <font color='darkgreen'>**sounddevice**</font>: mapping (bindings) en Python de la librería **PortAudio**
  (entrada/salida de audio multiplataforma)

- <font color='darkgreen'>**soundfile**</font>: carga y guardado de archivos de sonido


# Un reproductor básico

In [ ]:
# Reproductor básico

import numpy as np         # arrays    
import sounddevice as sd   # modulo de conexión con portAudio
import soundfile as sf     # para lectura/escritura de wavs

# leemos wav: data (array numpy con samples y SRATE)
# por defecto lee en formato dtype="float64" 
# En reproducción simple hace conversiones internas,
# pero en general haremos conversión explícita a np.float32, con dtype=np.float32
data, SRATE = sf.read('media/ex1.wav')  

# informacion de wav
print("\n\nInfo del wav " )
print("  Sample rate ", SRATE)  # leído del archivo
print("  Sample format: ", data.dtype)
print("  Num channels: ", len(data.shape))
print("  Len: ", data.shape[0])
  
# bajamos volumen: operación en numpy
data = data * 0.5

# a reproducir!
sd.play(data, SRATE)

# bloqueamos la ejecución hasta que acabe la reproducción
sd.wait()

# Buffering: procesamiento por **chunks**

El player anterior lee todos los datos de golpe y los envía al stream,
**bloqueando** el proceso de ejecución $\leadsto$ no se pueden manipular las muestras durante la reproducción

- Por ejemplo, no se puede modificar el volumen una vez arrancada la reproducción.



## Solución: envío por bloques (chunks)

<center>
<img src="media/chunks.png" width="50%" />
</center>


Los chunks son bloques (arrays de numpy) de tamaño prefijado que se leen/procesan/envían secuencialmente de uno en uno a sounddevice

In [ ]:
import numpy as np         # arrays    
import sounddevice as sd   # modulo de conexión con portAudio
import soundfile as sf     # para lectura/escritura de wavs
CHUNK = 2048   # tamaño CHUNK o bloque

# Leemos wav. Por defecto lee float64: no soportado por portAudio, 
# Convertimos directamente la conversion a float32
data, SRATE = sf.read('media/ex1.wav',dtype=np.float32)

# stream de salida
stream = sd.OutputStream( # creamos stream 
    samplerate = SRATE,            # frec de muestreo 
    blocksize  = CHUNK,            # tamaño del bloque
    channels   = len(data.shape))  # num de canales
stream.start() # arrancamos stream

prog = ['-','\\','|','/'] # para decorar
vol = 0.5      # volumen
numBloque = 0  # contador de bloques
end = False # para detección de últimol chunck 

while not(end): 
    # slice de data (no copia!). Si no hay suficientes samples, extrae los que queden
    bloque = data[numBloque*CHUNK : (numBloque+1)*CHUNK]
    
    if bloque.shape[0]<CHUNK: # ultimo bloque? -> se hace esta vuelta del bucle y terminamos
        end = True 
    
    bloque *= vol # modificamos volumen del bloque en cada vuelta del bucle!

    stream.write(bloque) # escribimos al stream de sounddevice
    
    numBloque += 1    
    print(f'\rProgreso: {prog[numBloque%4]}   bloque: {numBloque}',end='')

print(f'\nÚltimo bloque: {bloque.shape[0]} samples')
stream.stop()  # cerramos stream
stream.close() 


#### Interacción en tiempo real

Por ejemplo: modificar el volumen mientras suena

- Leeremos input de teclado 

- La lectura de teclado *sencilla* es **bloqueante**

In [ ]:
x = input("Nombre: ") # bloquea ejecución hasta pulsación de intro
print(f"Te llamas {x}")

### Opciones para **lectura no bloqueante en Python**:

- Con widgets de los notebooks (*ipywidgets*), a través de botones, sliders, etc 

    - Exige introducir bastante código adicional...
    - Poco útil fuera de los notebooks

- Con librerías específicas de Windows, Linux (no multiplataforma)

- Clase basada en la librería **pygame** (abre una ventana emergente que que lee input y lo devuelve con el método *getKey()*)

- Librería TKInter (similar al anterior, lo utilizaremos más adelante)

- Con una clase para trabajar directamente en terminal (multiplaforma): https://github.com/simondlevy/kbhit

#### En este notebook vamos a utilizar esta última por simplicidad... pero no funciona en los notebooks!!

- Utilizamos el clase KBHit del módulo kbhit (disponible en el CV o en https://github.com/simondlevy/kbhit)

- Pero ejecutamos fuera del notebook (como programa independiente). Para ello escribimos la celda en un archivo con la directiva 
    
    ```%%writefile file.py```

    guarda el código de esa celda en el archivo ```file.py```

- Luego ejecutamos ```file.py``` desde terminal o vs code    


## Player con control volumen: 

### Procesamiento por chunks + interacción de teclado

In [ ]:
%%writefile playerVol.py

import numpy as np         
import sounddevice as sd   
import soundfile as sf     
import kbhit # para lectura no bloqueante de teclado

CHUNK = 1024 # tamaño del chunk

data, SRATE = sf.read('media/ex1.wav',dtype=np.float32)

stream = sd.OutputStream(samplerate = SRATE, blocksize = CHUNK, channels = len(data.shape))
stream.start()

# para leer teclado
kb = kbhit.KBHit()

vol = 1.0
numBloque = 0 # contador de bloques/chunks
end = False # será true cuando el chunk esté incompleto o se pare la reproducción

while not(end): 
    bloque = data[numBloque*CHUNK : (numBloque+1)*CHUNK]
    if bloque.size<CHUNK: end = True # ultimo bloque?
    bloque *= vol # aplicamos volumen
    stream.write(bloque) # escribimos al stream

    if kb.kbhit():
        c = kb.getch()  # variacion de volumen/abortar
        if c != '':        
            if (c=='v'): vol= max(0,vol-0.05)
            elif (c=='V'): vol= min(1,vol+0.05)
            elif c in ['q','escape']: end = True 
            print(f"\rVol: {vol:.2f}     bloque: {numBloque}",end='')
    numBloque += 1
    
kb.set_normal_term()
stream.stop()
stream.close()

# Grabación básica

In [ ]:
%%writefile record.py


import numpy as np         
import sounddevice as sd   
import soundfile as sf     
import kbhit 

SRATE = 48000 # frecuencia de muestreo
CHUNK = 1024  # tamaño del bloque

# abrimos stream de entrada: InpuStream
stream = sd.InputStream(samplerate=SRATE, blocksize=CHUNK, dtype=np.float32, channels=1)
stream.start()

# buffer para grabación. 
# (0,1): vacio (tamaño 0), 1 canal
buffer = np.empty((0, 1), dtype="float32")

kb = kbhit.KBHit()
c = ''

print('Grabando. Pulsa \'q\' para terminar')
# bucle de grabación
while c != 'q': 
     # recogida de samples en array 
    bloque, _check = stream.read(CHUNK) # devuelve un par (samples,bool)    
    buffer = np.append(buffer,bloque) # en bloque[0] están los samples
    if kb.kbhit(): 
        c = kb.getch()
        print(c)
    

stream.stop() 
kb.set_normal_term()


# reproducción del buffer adquirido
c = input('Quieres reproducir [S/n]? ')
if c!='n':
    sd.play(buffer, SRATE)
    sd.wait()

# volcado a un archivo wav, utilizando la librería soundfile 
c = input('Grabar a archivo [S/n]? ')
if c!='n':    
    sf.write("rec.wav", buffer, SRATE)

stream.stop()
stream.close()


# Hebras de ejecución, CallBacks

En todos los ejemplos anteriores, la llamada `stream.write(...`

- sigue siendo "algo" <span style='color:darkgreen'>**bloqueante**</span>: bloquea la ejecución hasta que se completa el envío de datos al flujo

   $\leadsto$ tenemos control *a intervalos*: podemos interactuar entre
    envíos de bloques, p.e., para variar el volumen durante la
    reproducción

- En general la versión que tenemos ya es suficiente para muchas aplicaciones    

Una opción para tener más control: crear una nueva <font color='darkgreen'>**hebra de ejecución**</font> con la reproducción para no tener ningún bloqueo! 

- Aun así... la variación de volumen tiene efecto entre CHUNKS

### Sounddevice ya gestiona las hebras!

In [ ]:
# stream de salida con callBack
stream = sd.OutputStream(
  samplerate = SRATE, 
  channels = len(data.shape),
  blocksize = CHUNK, 
  callback = callback) # función callback se llama bajo demanda de chunks

$\leadsto$ se invoca a la función **callback** cuando el stream demanda nuevo audio para reproducir


## La función *callback*

Prototipo:

```python
callback(outdata,      # datos de salida
         frames,       # num nBloques a procesar por el stream = len de outdata
         time_info,    # estructura con current_frame_frameTime, etc
         status_flags) 
```             

En la práctica, lo esencial es: 

- *rellenar* `outdata` con los samples de salida (no crear un nuevo vector!) y

- tener cuidado con el *shape* del array que se copia a `outdata`: espera el formato  *(frames, channels)* 
    - En mono, numpy generar array con formato *(n,)* y hay que convertir a *(n,1)* con *reshape*:
    
        ```if (len(data.shape)==1): data = np.reshape(data,(data.shape[0],1))```

Menos esencial:

- `frames` el número de frames = long de `outdata` = `CHUNK`

-   `time_info`: contiene `input_buffer_adc_time`,
    `current_frame_time` y `output_buffer_dac_time` (ver documentación de PortAudio)
-   `status_flags` (ver documentación de PortAudio)



# Reproductor con callback


In [ ]:
%%writefile playCB.py


import numpy as np         
import sounddevice as sd   
import soundfile as sf     
import kbhit               

CHUNK = 2048
data, SRATE = sf.read('media/ex1.wav',dtype=np.float32)

# para archivos mono, devuelve un array de la forma data.shape = (n,)
# para rellenar el array outdata del callback se necesita hacer explícito el número de canales
# convertir el data.shape  (n,) -> (n,1) 
if (len(data.shape)==1): data = np.reshape(data,(data.shape[0],1))
# otra forma data = data.reshape(-1, 1)

# info del wav
print(f"SRATE: {SRATE}   Format: {data.dtype}   Channels: {len(data.shape)}    Len: {data.shape[0]}")


# contador de frames, global
current_frame = 0
def callback(outdata, frames, time, status):
    global current_frame       # para actualizarlo en cada callBack
    if status: print(status)

    # ojo, este print es muy lento... puede provocar underruns
    print(f"\rNum Bloque: {current_frame//CHUNK}  frame: {current_frame}", end='') 

    # escribimos los samples correspondientes en el outdata que viene dado
    bloque = data[current_frame : current_frame+CHUNK]
    # print(f'shape[0]: {bloque.shape},   pad: {CHUNK-bloque.shape[0]}')

    # tamaño del blque leido
    chunksize = bloque.shape[0]

    outdata[:chunksize] = bloque            
    # es una forma EFICINTE de rellenar outdata, copiando los samples del bloque 
    # Es similar a 
    #   for i in range(chunksize): outdata[i] = bloque[i]
    # pero este es for es demasiado lento (es un for de python)-> underruns!!

    # NO funcionaría hacer outdata = data[current_frame:current_frame + chunksize]
    # compartiría referencias (objetos array de numpy)
    # outdata viene dado y hay que rellenar su contenido

    
    if chunksize < frames: # ha terminado?
        print('fin')
        outdata[chunksize:] = 0 # rellenamos con 0's el resto de outdata
        raise sd.CallbackStop()

    # actualizamos frame con los frames procesados    
    current_frame += chunksize


# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, channels=len(data.shape),
    callback=callback, blocksize=CHUNK)
stream.start()

# con esto empezaría a reproducir, pero si la hebra ppal termina, se para la reproducción
# Necesitamos mantener viva esta hebra

kb = kbhit.KBHit()
c = '' 
while c not in ['q']: # para bloquear ejecución en la hebra ppal mientras reproduce
    if kb.kbhit():
        c = kb.getch()


# limpieza
kb.normal_term()
stream.stop()
stream.close()

## Entendiendo el array `outdata`?

Hay que copiar `CHUNK` muestras de `data` en`outdata`, rellenando el `outdata` que viene dado, sin generar un nuevo array

In [6]:
# mini ejemplo para ver la copia de muestras

import numpy as np
CHUNK = 5


# next es un # "iterador" que va dando sucesivos chunks
# en cada llamada
current_frame = 0
def next(outdata): 
    global current_frame 

    # nuevo bloque de tamaño CHUNK (si queda suficiente)
    bloque = data[current_frame:current_frame+CHUNK]

    # vemos lo que ha cogido
    size = bloque.shape[0]

    # y lo ponemos en outdata, al principio
    outdata[:size] = data[current_frame:current_frame+size]
    current_frame += size

    # si out no está completo, relleamos con 0s
    if size<CHUNK:
        outdata[size:] = 0
        print('FIN')

# data con 6 eltos [0..5]
data = np.arange(6,dtype=np.float32)

# outdata de tamaño CHUNK (5), con ceros
outdata = np.zeros(CHUNK)

print('Inicialmente (antes de llamar a next)')
print('data:',data)
print('outdata: ',outdata)


Inicialmente (antes de llamar a next)
data: [0. 1. 2. 3. 4. 5.]
outdata:  [0. 0. 0. 0. 0.]


In [8]:

# ahora ejecutar sucesivas veces
# outdata es siempre el mismo array que se va rellenando con sucesivos slices (copias)
next(outdata)
print(outdata)


FIN
[5. 0. 0. 0. 0.]


# Grabación con callBack (incluso más fácil)

In [ ]:
# buffer para acumular grabación.
# (0,1): de tamao 0 (vacío), y con 1 canal 
buffer = np.empty((0, 1), dtype=np.float32)

def callback(indata, frames, time, status):
    global buffer
    # concatenamos indata al buffer
    buffer = np.append(buffer,indata)

# stream de entrada con callBack
stream = sd.InputStream(
    samplerate=SRATE, dtype=np.float32,  channels=CHANNELS,
    blocksize=CHUNK, callback=callback)
